In [1]:
import pandas as pd
import datetime
import nltk
import spacy

import re
from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


data = pd.read_json("../input/arxivdataset/arxivData.json")
print(list(data))
data.shape
data.head()
data.isnull().sum()

['author', 'day', 'id', 'link', 'month', 'summary', 'tag', 'title', 'year']


author     0
day        0
id         0
link       0
month      0
summary    0
tag        0
title      0
year       0
dtype: int64

In [2]:
# ### smaller data. subset: 
# data = data.loc[data["year"]>2016]

In [3]:
data.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


In [4]:
data = data.dropna().reset_index(drop=True)
data.isnull().sum()

author     0
day        0
id         0
link       0
month      0
summary    0
tag        0
title      0
year       0
dtype: int64

In [5]:
data["year"] = pd.to_datetime(data["year"])

# data = data.sort_values(by="year") # disable for easier comparison
data.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,1970-01-01 00:00:00.000002018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,1970-01-01 00:00:00.000002016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,1970-01-01 00:00:00.000002016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,1970-01-01 00:00:00.000002017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,1970-01-01 00:00:00.000002017


In [6]:
data_abstract = data['summary'].str.lower()
data_abstract.head()

0    we propose an architecture for vqa which utili...
1    recent approaches based on artificial neural n...
2    we introduce the multiresolution recurrent neu...
3    multi-task learning is motivated by the observ...
4    we present milabot: a deep reinforcement learn...
Name: summary, dtype: object

In [7]:
x= len(data_abstract)
print(x)

41000


In [8]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

##ORIGL
# def cleaning(doc):
#     # Lemmatizes and removes stopwords
#     # doc needs to be a spacy Doc object
#     txt = [token.lemma_ for token in doc if not token.is_stop]
#     # Word2Vec uses context words to learn the vector representation of a target word,
#     # if a sentence is only one or two words long,
#     # the benefit for the training is very small
#     if len(txt) > 2:  
#         return ' '.join(txt)

##ALT - speedup: 

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    if len(doc)>2:
        txt = [token.lemma_ for token in doc if not token.is_stop]
        # Word2Vec uses context words to learn the vector representation of a target word,
        # if a sentence is only one or two words long,
        # the benefit for the training is very small
        if len(txt) > 2:
            return ' '.join(txt)

In [9]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in data['summary'])

In [10]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 2.58 mins


In [11]:
data_clean = pd.DataFrame({'clean': txt})
data_clean = data_clean.dropna().drop_duplicates()
data_clean.shape

(40955, 1)

In [12]:
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in data_clean['clean']]

* https://stackoverflow.com/questions/35716121/how-to-extract-phrases-from-corpus-using-gensim

In [13]:
print(sent[0])

['propose', 'architecture', 'vqa', 'utilize', 'recurrent', 'layer', 'generate', 'visual', 'textual', 'attention', 'memory', 'characteristic', 'propose', 'recurrent', 'attention', 'unit', 'offer', 'rich', 'joint', 'embedding', 'visual', 'textual', 'feature', 'enable', 'model', 'reason', 'relation', 'part', 'image', 'question', 'single', 'model', 'outperform', 'place', 'winner', 'vqa', 'dataset', 'perform', 'margin', 'current', 'state', 'art', 'ensemble', 'model', 'experiment', 'replace', 'attention', 'mechanism', 'state', 'art', 'model', 'implementation', 'increase', 'accuracy', 'case', 'recurrent', 'attention', 'mechanism', 'improve', 'performance', 'task', 'require', 'sequential', 'relational', 'reasoning', 'vqa', 'dataset']


In [14]:
phrases = Phrases(sent, min_count= 20, progress_per=10000) # min_count=30
bigram = Phraser(phrases)

In [15]:
sentences = bigram[sent]

In [16]:
### get trigrams, by including found bigrams/phrases
phrases2 = Phrases(sent, min_count= 20, progress_per=10000) # min_count=30
trigram = Phraser(phrases2)

sentences = trigram[sentences]

In [17]:
sentences[0]

['propose',
 'architecture',
 'vqa',
 'utilize',
 'recurrent',
 'layer',
 'generate',
 'visual_textual',
 'attention',
 'memory',
 'characteristic',
 'propose',
 'recurrent',
 'attention',
 'unit',
 'offer',
 'rich',
 'joint',
 'embedding',
 'visual_textual',
 'feature',
 'enable',
 'model',
 'reason',
 'relation',
 'part',
 'image',
 'question',
 'single',
 'model',
 'outperform',
 'place',
 'winner',
 'vqa',
 'dataset',
 'perform',
 'margin',
 'current_state',
 'art',
 'ensemble',
 'model',
 'experiment',
 'replace',
 'attention_mechanism',
 'state_art',
 'model',
 'implementation',
 'increase',
 'accuracy',
 'case',
 'recurrent',
 'attention_mechanism',
 'improve_performance',
 'task',
 'require',
 'sequential',
 'relational',
 'reasoning',
 'vqa',
 'dataset']

In [18]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

50696

In [19]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['model',
 'method',
 'propose',
 'algorithm',
 'base',
 'image',
 'problem',
 'approach',
 'datum',
 'network']

In [20]:
import multiprocessing

from gensim.models import Word2Vec

In [21]:
cores = multiprocessing.cpu_count()

In [22]:
w2v_model = Word2Vec(min_count=15,  # min_count=20,
                     window=5,  # window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [23]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.35 mins


In [24]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=25, # epochs=30,
                report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 11.45 mins


In [25]:
w2v_model.init_sims(replace=True)

In [26]:
w2v_model.wv.most_similar(positive=["software"])

[('software_development', 0.6366830468177795),
 ('developer', 0.5696251392364502),
 ('java', 0.5335867404937744),
 ('lifecycle', 0.5296437740325928),
 ('documentation', 0.5234754681587219),
 ('software_engineering', 0.5078887939453125),
 ('javascript', 0.49422377347946167),
 ('browser', 0.49050840735435486),
 ('gui', 0.4880084991455078),
 ('open_source', 0.4834585189819336)]

In [27]:
w2v_model.wv.most_similar(positive=["engineering"])

[('civil', 0.5638142824172974),
 ('industrial_application', 0.5010552406311035),
 ('biology', 0.4768846929073334),
 ('science_engineering', 0.4629703760147095),
 ('social_science', 0.44419991970062256),
 ('computer_scientist', 0.4439702332019806),
 ('discipline', 0.4436374008655548),
 ('software_engineering', 0.44242459535598755),
 ('engineer', 0.43597304821014404),
 ('medicine', 0.43518027663230896)]

In [28]:
w2v_model.wv.similarity("technology", 'era')

0.46913132

In [29]:
w2v_model.wv.doesnt_match(['era', 'approach', 'technology'])

/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'approach'

In [30]:
w2v_model.wv.most_similar(positive=["technology", "era"], negative=["approach"], topn=5)

[('rapid_growth', 0.6205344200134277),
 ('advent', 0.6116254329681396),
 ('revolution', 0.6055727601051331),
 ('deluge', 0.6000558137893677),
 ('explosive', 0.5954705476760864)]

In [31]:
w2v_model.wv.most_similar(positive=["machine_learning"])

[('machine_learn', 0.7673787474632263),
 ('bioinformatics', 0.5214214324951172),
 ('datum_mining', 0.501287579536438),
 ('science_engineering', 0.49420303106307983),
 ('bioinformatic', 0.4727438688278198),
 ('social_science', 0.47272995114326477),
 ('deep_learning', 0.46382689476013184),
 ('computational_linguistic', 0.4500200152397156),
 ('informatic', 0.44544899463653564),
 ('renew_interest', 0.4432750344276428)]

In [32]:
w2v_model.wv.most_similar(positive=["ai"])

[('artificial_intelligence', 0.689173698425293),
 ('ethic', 0.5939954519271851),
 ('agi', 0.5767300128936768),
 ('intelligent_agent', 0.5041818618774414),
 ('artificial_life', 0.5033911466598511),
 ('cognitive_science', 0.5016610622406006),
 ('superhuman', 0.5014196634292603),
 ('ethical', 0.4807817339897156),
 ('intelligence', 0.4680931568145752),
 ('assistant', 0.46774715185165405)]

In [33]:
w2v_model.wv.most_similar(positive=["artificial_intelligence","ai",'machine_learning','machine_learn'])

[('computer_science', 0.5688163638114929),
 ('renew_interest', 0.558725118637085),
 ('science_engineering', 0.5564792156219482),
 ('cognitive_science', 0.5556671619415283),
 ('endeavor', 0.5555830001831055),
 ('computational_linguistic', 0.5461016893386841),
 ('academia_industry', 0.5415195822715759),
 ('discipline', 0.5376780033111572),
 ('revolutionize', 0.5268438458442688),
 ('social_science', 0.52638840675354)]

In [34]:
w2v_model.wv.most_similar(positive=["attention_mechanism"])

[('soft_attention', 0.608681857585907),
 ('attend', 0.6000659465789795),
 ('attention', 0.5988219976425171),
 ('attentive', 0.5814162492752075),
 ('attentional', 0.5723384618759155),
 ('selectively', 0.5515202283859253),
 ('visual_textual', 0.47868841886520386),
 ('seq_seq', 0.4767484962940216),
 ('recurrent', 0.4739079773426056),
 ('abstractive', 0.4722852408885956)]

In [35]:
w2v_model.wv.most_similar(positive=["embedding"])

[('embed', 0.6614126563072205),
 ('vector_space', 0.571323037147522),
 ('representation', 0.569983720779419),
 ('transe', 0.5257548093795776),
 ('entity_relation', 0.5211679935455322),
 ('word_embedding', 0.520942211151123),
 ('skip_gram', 0.5195310115814209),
 ('embe', 0.5083346366882324),
 ('vec', 0.5074207186698914),
 ('word_vec', 0.49638521671295166)]